In [3]:
data_path = '/home/yigit/projects/is/data/'
folder_path = f'{data_path}data_raw/'

time_steps = 400

In [6]:
import numpy as np
import matplotlib.pyplot as plt


def process_file(file_path):
#     if '7_2_states.npy' in file_path:
#         temp = np.load(file_path)
#         np.save(file_path, temp[1:])
    states = np.load(file_path, allow_pickle=True)
    ego_states = np.zeros_like(states)
    
    # final pose of the traj is the goal
    start_pose, goal_pose = states[0, :2], states[-1, :2]
    for i, state in enumerate(states):
        if i==len(states)-2:
            break
        ego_states[i, :2] = goal_pose - state[:2]  # dg
        ego_states[i, 2:4] = state[2:4]  # dp
#     ego_states[i, -2:] = [0, 0]  # last vel is 0
        next_state = states[i+1]
        print(next_state[-2] - state[-2])
        break
        vx, vy = (next_state[:2] - state[:2]) / (next_state[-2] - state[-2])
    return start_pose, goal_pose, ego_states


def interpolate(states):
    t_orig, d = states.shape
    
    t = np.linspace(0, 1, time_steps)
    tp = np.linspace(0, 1, t_orig)
    
    cont_states = np.zeros((time_steps, d))
    
    for i in range(d):
        cont_states[:, i] = np.interp(t, tp, states[:, i])
        
    return cont_states


def plot_trajectories(trajectories):
    num_trajs = trajectories.shape[0]
    for i in range(num_trajs):
        plt.plot(trajectories[i, :, 0], trajectories[i, :, 1])
    
    print(f'Number of Trajectories: {num_trajs}')
    plt.show()

In [7]:
import os


trajectories = []
for filename in os.listdir(folder_path):
    f = os.path.join(folder_path, filename)
    
    # checking if it is a file
    if os.path.isfile(f) and f.endswith('npy'):
        start, goal, ego_states = process_file(f)
        cont_states = interpolate(ego_states)
#         print(filename)
#         print(ego_states)
        np.save(f"{data_path}/processed_states/{filename.strip('_states.npy')}.npy", cont_states)
        trajectories.append(cont_states)

trajectories = np.array(trajectories)
plot_trajectories(trajectories)

100855112


TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [7]:
print(trajectories[0].shape)
np.savetxt('/home/yigit/Desktop/0.txt', trajectories[0])

(400, 6)
